- Se debe generar en aws un bash con el script de python que se desea ejecutar

- Después, mediante ssh se ejecuta el siguiente comando:

<center> ssh -i Deliveryrobotpassword.pem ubuntu@18.116.62.44 "bash ~/bash_test.sh" 

In [78]:
%%writefile 'script_to_send_images_and_get_tflite_model.py'

import paramiko
from scp import SCPClient
from datetime import datetime
import time
import os

def create_SSH_Client(host, port, user, password_name, password_path):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(host, port, user, password_name, key_filename=password_path)
    return client

ip_servidor = "3.142.240.64"
ruta_clave = "../../../proyectos_ai/delivery_robot/Deliveryrobotpassword.pem"

while True:
    fecha = datetime.now()
    if fecha.hour == 15:
        #print("Entrando")
        #se cargan las imaǵenes al servidor
        ssh = create_SSH_Client(ip_servidor, 22, "ubuntu","Deliveryrobotpassword.pem", ruta_clave)
        scp = SCPClient(ssh.get_transport())
        scp.put('../Images', recursive=True,
                remote_path='/home/ubuntu/delivery_robot_project/static/')
        #se cierra la conexión al terminar
        ssh.close()
        #se ejecuta el script en aws que permite reentrenar el modelo de tensorflow
        # y genera un nuevo modelo en tensorflowlite
        os.system('ssh -i '+ruta_clave+' ubuntu@'+ip_servidor+' "sudo bash /home/ubuntu/bash_entrenar.sh"')
        
        #Después de entrenador y generado el tflite en el servidor, nos traemos el .tflite para que
        #funcione en la interfaz de tkinter
        os.system('scp -i '+ruta_clave+' ubuntu@'+ip_servidor+':~/delivery_robot_project/static/model_tflite_'+str(fecha.month)+'_'+str(fecha.day)+'.tflite'+' ../data/')
        #se eliminan las imágenes del servidor
        break
        time.sleep(60*60) #se para la ejecución por una hora


Writing script_to_send_images_and_get_tflite_model.py


In [77]:
%run "script_to_send_images_and_get_tflite_model.py"

### script para entrenar el modelo en aws
- este script se ejecuta en aws

In [12]:
%%writefile "script_para_entrenar_en_aws.py"

import tensorflow as tf
import json
import os
import numpy as np
from datetime import datetime

batch_size = 64
img_height = 145
img_width = 145
data_root='./Images'
print("datos de entrenamiento")
data_train = tf.keras.preprocessing.image_dataset_from_directory(str(data_root),
                                                               seed=123,
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size,
                                                               label_mode='categorical')

#definimos el model_augmentation que transforma las imágenes de entrada
model_augmentation = tf.keras.Sequential(name="augmentation_model")
model_augmentation.add(tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"))
model_augmentation.add(tf.keras.layers.experimental.preprocessing.RandomRotation(.1, fill_mode="constant"))
model_augmentation.add(tf.keras.layers.experimental.preprocessing.RandomZoom(.2, fill_mode="constant"))
#mapeamos los datos de entrada
map_data_train = data_train.map(lambda x, y: (model_augmentation(x), y))
#cargamos el modelo entrenado
modelo_entrenado = tf.keras.models.load_model("./models_finalversionmap15.0version145x145")
#entrenamos el modelo
hist_model = modelo_entrenado.fit(map_data_train, epochs=10)
#guardamos un json que contiene el historial de entrenamiento
open("./resultados_entrenamiento.json", "w").write(json.dumps(hist_model.history))
#gurdamos el modelo entrenado
#por ahora no sobreescribimos el modelo, lo guardamos con el mes y el dia
fecha = datetime.now()
modelo_entrenado.save("./modelo_reentrenado_"+str(fecha.month)+"_"+str(fecha.day))
#generamos el modelo en formato tensorflowlite a partir del modelo reentrenado
converter = tf.lite.TFLiteConverter.from_keras_model(modelo_entrenado)
tflite = converter.convert()
#guardamos el modelo de tensorflowlite (tambien por mes y dia)
open("./model_tflite_"+str(fecha.month)+"_"+str(fecha.day)+".tflite", "wb").write(tflite)


Writing script_para_entrenar_en_aws.py


In [66]:
ip_servidor = "3.142.240.64"
ruta_clave = "../../../proyectos_ai/delivery_robot/Deliveryrobotpassword.pem"

os.system('ssh -i '+ruta_clave+' ubuntu@'+ip_servidor+' "sudo bash /home/ubuntu/bash_entrenar.sh"')

0

In [71]:
os.system('scp -i '+ruta_clave+' ubuntu@'+ip_servidor+':~/delivery_robot_project/static/model_tflite_'+str(fecha.month)+'_'+str(fecha.day)+'.tflite'+' ./')

0